# this file can be used to add new mappings in case hardware_instances obtains new stuff. All changes are indicated

In [1]:
# import builtins

# # Save the original print function
# original_print = print

# # Override the print function
# def custom_print(*args, **kwargs):
#     new_args = []
#     for arg in args:
#         if isinstance(arg, float):
#             # Customize the format here
#             new_args.append(f'{arg:.10f}')
#         else:
#             new_args.append(arg)
#     original_print(*new_args, **kwargs)

# # Override the built-in print with the custom one
# builtins.print = custom_print

In [2]:
import pandas as pd
import os
import gzip
import pickle
import openai
import torch.nn.functional as F
import torch
import re
import copy
from tqdm import tqdm
from openai import OpenAI
client = OpenAI()

In [3]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
# enc.encode("<|endoftext|>",allowed_special={'<|endoftext|>'})
enc.encode(" #")

[674]

In [4]:
def get_openai_response(prompt):
    tokens = 4000
    model="gpt-4-turbo-preview"
    # model="gpt-3.5-turbo-0125"
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": """
The current date is Tuesday, September 10th 2023, 10:30am
You are a helpful assistant.
""".strip()
            },
            {
                "role": "user",
                "content": prompt
            }
            ],
        temperature=0,
        max_tokens=tokens,
        top_p=1,
        # logit_bias={"674": -1.5},
    )
    choice = response.choices[0]

    text = choice.message.content


    return text

In [5]:
from hardware_mapping import map_hardware_to_table
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))

### new

In [6]:
hardware_instances = hardware_instances[hardware_instances["hardware_mapped"] == "not found"]

### new

In [7]:
hardware_instances

,date,hardware,hardware_mapped
1,2011-01-21 18:36:10,gt330m,not found
2,2011-01-21 18:36:10,gtx 460,not found
3,2011-01-21 18:36:10,8800gtx,not found
8,2011-02-27 21:32:08,6-year old windows desktop,not found
12,2011-02-27 21:32:08,n900 phone,not found
...,...,...,...
53380,2023-10-25 23:42:40,s9 firmware,not found
53381,2023-10-25 23:42:40,r4 tar.gz files,not found
53382,2023-10-25 23:42:40,zynq c41,not found
53384,2023-10-26 08:40:14,avalonminer 1166pro-72,not found


In [8]:


# add row index
hardware_instances = hardware_instances.reset_index(drop=True)
hardware_instances["row_index"] = hardware_instances.index


In [9]:
unique_values = hardware_instances["hardware"].value_counts().index.tolist()
unique_values.sort()

for a in unique_values:
    print(a)

"oc dual-x"
#1 computer
#11
#12 2-conductor with ground wire cable
#13
#2 computer
#6/#8
$16 holmes personal fans
$1600 laptop
$400 gpus
$400 rig
$500 pc
* .batstart batch:cgminer.exe
-qt client
-v -w256 -f0
.bat or .cmd files
.net 3.5 framework
.net 4 framework
.net framework
.net program widget thingie
.vbs script
/dev/ttyacm0
/etc/init.d/cgminer
/etc/x11... file
0 asic hashboard
0.5 and 1.0 thermal pads
0.5 bamt installation
0.75mm2 cables (21awg)
021d75d0
06 width: 64 bits clock: 33mhz
06748be7
08 macbook pro
08volts regulator
1 1050w psu
1 120v 20 amp
1 5870
1 5970
1 bfl sc 60
1 board s9
1 card
1 chip furies
1 corsair ax1200 (1200w 6(6p+2p)1 old 160gb sata hd
1 fan
1 gb ddr2
1 gb of video
1 gh/s
1 gig of ram
1 gpu thread
1 ground
1 hdd
1 megawatt town commercial license
1 pcie (6pin) connector
1 ph/s miner
1 phase 200 amp panel
1 pole breaker
1 pump
1 s7 (3.2 th/s)
1 sappire 5850 xtreme
1 single gpu card
1 ssd
1 th image
1 th/s dragonminer
1 th/s miner
1 ths sino sigma asic miner


In [10]:
possible_hardware = pd.read_csv('../../hardwarelist/hardware_merged.csv')["hardware_name"].sort_values().tolist()
possible_hardware = [x.lower() for x in possible_hardware]

In [11]:
possible_hardware[:3]

['3410', '4350', '4550']

In [12]:
def have_common_chunk(str1, str2):
    # Iterate through the first string
    for i in range(len(str1) - 1):  # Subtract 1 to avoid index out of range
        # Extract a 2-letter chunk from the first string
        chunk1 = str1[i:i+2]
        
        # Iterate through the second string
        for j in range(len(str2) - 1):  # Subtract 1 to avoid index out of range
            # Extract a 2-letter chunk from the second string
            chunk2 = str2[j:j+2]
            
            # Compare the chunks
            if chunk1 == chunk2:
                return True  # Return True if a common chunk is found

### new: changed to mappings2 and split up code into 2 cells

In [13]:
chunk_length = 50
num_chunks = len(unique_values) // chunk_length + (1 if len(unique_values) % chunk_length != 0 else 0)
num_chunks

254

In [14]:


# Iterate through the lines chunk_length at a time
# for i in tqdm(range(num_chunks)):
for i in range(num_chunks):
    
    # Calculate start and end indices for slicing
    start_index = i * chunk_length
    end_index = start_index + chunk_length
    chunk = unique_values[start_index:end_index]
    text = "\n".join(chunk).replace("  "," ")

    if not text:  # Make sure the chunk is not empty
        continue
    # check if the file already exists	
    if os.path.exists(f"mappings2/hardware_mapping_{i}.txt"):
        continue

    print("processing chunk",i)

    # temp_hardware_lines = "\n".join(possible_hardware)

    # for each hardware name, check if there is at least one string in the chunk with at least 2 letters in common
    # temp_hardware_lines = []
    # for hardware in possible_hardware:
    #     for string in chunk:
    #         if len(set(hardware).intersection(set(string))) >= 2:
    #             temp_hardware_lines.append(hardware)
    #             break

    # for each hardware name, check if there is at least one string in the chunk with at least a 2-gram in common
    temp_hardware_lines = []
    for hardware in possible_hardware:
        for string in chunk:
            if have_common_chunk(hardware, string):
                temp_hardware_lines.append(hardware)
                break

    print(len(temp_hardware_lines))
    temp_hardware_lines = "\n".join(temp_hardware_lines)

    prompt = f"""
User:

Here is a list of hardware that is used for mining bitcoin:

{temp_hardware_lines}



You will be given a collection of strings. Some of them correspond to entries in the list above, while others are unrelated.
Your task is to create a python dictionary that maps the strings from the collection to the names in the list above.
If a string is unclear or not in the table of mining hardware, write "unknown" for the hardware_index value. The names don't need to match exactly.

Example input:
```
sapphire 7950 vapor x
1th/s s2 antminer
1 th mining system
bpmc "blue fury" 2.7 gh/s usb miner v1
18 btc garden v2
antminer s5
block erupter asics
14th s9 of nov.2017 batch
```

Example output:
```python
mapping = {"{"}
    "sapphire 7950 vapor x": "7950",
    "1th/s s2 antminer": "antminer s2",
    "1 th mining system": "unknown",
    "bpmc \\"blue fury\\" 2.7 gh/s usb miner v1": "blue fury",
    "btc garden am-v1", # Assuming a typo or version difference, closest match.
    "antminer s5": "antminer s5",
    "block erupter asics": "unknown",
    "14th s9 of nov.2017 batch": "antminer s9",
{"}"}
```

It is very important that the mapping's inputs correspond exactly to the strings in the collection, including any typo that might be present in the string.
If none of the strings match any hardware names, then write "unknown" for all of them.
In the rare cases where a string can be mapped to a hardware name but the mapping is questionable, write a comment on the same line.

Write the mapping for this collection of strings:
{text}



Assistant:
Sure! Here is the mapping for the collection of strings:
    """.strip()


    print(prompt)
    # print(temp_hardware_lines)
    # print(text)
    print("\n"*10)

    response = get_openai_response(prompt)


    print(response)
    print("\n"*10)

    # save to file
    with open(f"mappings2/hardware_mapping_{i}.txt", "w") as file:
        file.write(response)

    # break






processing chunk 38
276
User:

Here is a list of hardware that is used for mining bitcoin:

3410
4350
4550
4570m
4650
4670
4770
4850
4870
4890
5450
5550
5570
5650
5670
5750
5770
5830
5850
5870
5970
6310m
6450
6490m
6570
6670
6750
6790
6850
6870
6950
6970
6990
7750
7770
7850
7870 xt
7970
8400 gs
8800 gts
8800gt
8800gts
9400gt
9400m
9500gt
9600gso
9600gso512
9600gt
9800gt
9800gt ee
9800gtx
9800gtx+
9800gx2
aisen a1 pro
antminer r4
antminer s1
antminer s11
antminer s17
antminer s17 pro
antminer s17+
antminer s17e
antminer s19
antminer s19 hydro
antminer s19 pro
antminer s19 pro hyd
antminer s19 pro+ hyd
antminer s19 xp
antminer s19 xp hyd
antminer s19j
antminer s19j pro
antminer s19j pro+
antminer s19k pro
antminer s2
antminer s21
antminer s21 hyd
antminer s3
antminer s3
antminer s3+
antminer s4
antminer s5
antminer s5
antminer s5+
antminer s7
antminer s7
antminer s7-ln
antminer s9
antminer s9
antminer s9 hydro
antminer s9 se
antminer s9i
antminer s9j
antminer s9k
antminer t17
antminer t1

```python
mapping = {
    "5hdd's": "unknown",
    "5k amps of primary switchgear": "unknown",
    "5k watt 220v step up converter": "unknown",
    "5kw battery": "unknown",
    "5m 3/4\" tubing": "unknown",
    "5port switch": "unknown",
    "5t": "unknown",
    "5th card": "unknown",
    "5th psu": "unknown",
    "5th/s miner": "unknown",
    "5v 2.1 10w power supply": "unknown",
    "5v 20a psu": "unknown",
    "5v 2a power supply": "unknown",
    "5v 3 amp adapter": "unknown",
    "5v 40 amp power supply": "unknown",
    "5v 4a psu brick": "unknown",
    "5v 4pin": "unknown",
    "5v adapter": "unknown",
    "5v fan/hdd power cable": "unknown",
    "5v fans": "unknown",
    "5v hub": "unknown",
    "5v power supply": "unknown",
    "5v psu": "unknown",
    "5v usb fan": "unknown",
    "5v/4a hubs": "unknown",
    "5vdc wall wart psu": "unknown",
    "5x 7990 hd": "unknown", # No exact match, but refers to a specific model of GPU not listed.
    "5x pci e2x sapphire 100311sr radeon 

### new: changed file name and dir name

In [15]:
dict_string = ""
dir_name = "mappings2"
files_count = len(os.listdir(dir_name))

for (i,file) in enumerate(os.listdir(dir_name)):
    dict_string += "mapping" + str(i) + " = {\n"

    content = open(f"{dir_name}/{file}").read()

    relevant_lines = content.split("```")[1].split("\n")[2:-2]

    comment = content.split("```")[2].strip()

    for line in relevant_lines:
        # # check if there is a hashtag before the colon :
        # if line.split(":")[0].__contains__("#"):
        #     pass
        # else:
        #     # if a line has no comma at the end, add it
        #     if not line.split(":")[1].__contains__(","):
        #         print(line)
        #         if line.split(":")[1].__contains__("#"):
        #             # add a comma before the comment
        #             line = line.split(":")[0] + ": " + line.split(":")[1].split("#")[0].strip() + " # " + line.split("#")[1]
        #         else:
        #             line = line + ","


        dict_string += line + "\n"

    # add the comment
    if comment != "":
        dict_string = dict_string[:-1] + f"# {comment}\n"

    dict_string += "}" + "\n\n"


code = f"""
{dict_string}
def map_hardware_to_table2(string):
    res = "not found"
    for i in range(0, {files_count}):
        if string in globals()["mapping" + str(i)]:
            res = globals()["mapping" + str(i)][string]
            break
    return res
""".strip()

# save to file
with open("hardware_mapping2.py", "w") as file:
    file.write(code)

# don't run stuff below

In [ ]:
from hardware_mapping import map_hardware_to_table
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))

In [ ]:
hardware_instances.head(10)

In [ ]:
len(hardware_instances)

In [ ]:
efficiency = pd.read_csv('../../hardwarelist/hardware_merged.csv') # columns: hardware_name,Mhash/J
efficiency = efficiency.rename(columns={"hardware_name":"hardware_mapped"})

efficiency.head(2)

In [ ]:
joined = hardware_instances.merge(efficiency, on="hardware_mapped", how="left")
joined = joined.dropna(subset=["Mhash/J"])
joined.head(4)

In [ ]:
print(len(joined))

In [ ]:
table = joined[["date","hardware_mapped","Mhash/J"]]
table = table.rename(columns={"hardware_mapped":"hardware_name"})
table["Mhash/J"] = table["Mhash/J"].astype(float).map(lambda x: x/1000000).map(lambda x: f"{x:.10f}")
table = table.rename(columns={"Mhash/J":"TH/J"})
table = table.sort_values("date")
table = table.reset_index(drop=True)
table.head(8)

In [ ]:
table.to_csv("hardware_instances_with_efficiency.csv", index=False)